In [ ]:
# clone from github
# git clone https://github.com/CambioML/pykoi.git /home/ec2-user/SageMaker/pykoi

In [ ]:
# Option 1:
# use github code as source
import sys
sys.path.append("/home/ec2-user/SageMaker/pykoi")
!pip install poetry
!pip install -q nest_asyncio
!poetry install --no-root
!pip3 install torch --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# # Option 2:
# # pip install pykoi into site-packages
# !pip install -q nest_asyncio
# !pip install pykoi
# !pip3 install torch --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!nvcc --version

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pykoi
import nest_asyncio
nest_asyncio.apply()

In [ ]:
print("create model 1...")
hf_model_1 = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="tiiuae/falcon-rw-1b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)

print("create tokenizer 1...")
hf_tokenizer_1 = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="tiiuae/falcon-rw-1b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)

###################################################################################
# Creating a Huggingface model tiiuae/falcon-7b (EC2 g4.2xlarge with 100GB space) #
###################################################################################
print("create model 2...")
hf_model_2 = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="tiiuae/falcon-7b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)

print("create tokenizer 2...")
hf_tokenizer_2 = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="tiiuae/falcon-7b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)

#########################################################################################
# Creating a Huggingface model databricks/dolly-v2-3b (EC2 g4.2xlarge with 100GB space) #
#########################################################################################
print("create model 3...")
hf_model_3 = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="databricks/dolly-v2-3b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)

print("create tokenizer 3...")
hf_tokenizer_3 = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="databricks/dolly-v2-3b",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
)


In [ ]:
# pass in a list of models to compare
model_name = ["falcon-rw-1b", "falcon-7b", "dolly-v2-3b"]
models = [hf_model_1, hf_model_2, hf_model_3]
tokenizers = [hf_tokenizer_1, hf_tokenizer_2, hf_tokenizer_3]

models_list = [
    pykoi.chat.llm.huggingface.HuggingfaceModel.create(
        model=model, tokenizer=tokenizer, name=name, max_length=100
    )
    for model, tokenizer, name in zip(models, tokenizers, model_name)
]
chatbot_comparator = pykoi.Compare(models=models_list)

In [ ]:
app = pykoi.Application()
app.add_component(chatbot_comparator)
app.run()
